<h1>2장 토큰과 임베딩</h1>
<i>LLM 구축의 핵심적인 부분인 토큰과 임베딩을 살펴 봅니다.</i>


<a href="https://github.com/rickiepark/handson-llm"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rickiepark/handson-llm/blob/main/chapter02.ipynb)

---

이 노트북은 <[핸즈온 LLM](https://tensorflow.blog/handson-llm/)> 책 2장의 코드를 담고 있습니다.

---

<a href="https://tensorflow.blog/handson-llm/">
<img src="https://tensorflow.blog/wp-content/uploads/2025/05/ed95b8eca688ec98a8_llm.jpg" width="350"/></a>


---

💡 **NOTE**: 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.

---

In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 63.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.2
    Uninstalling transformers-4.55.2:
      Successfully uninstalled transformers-4.55.2


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

# LLM 다운로드하고 실행하기

빠른 추론을 위해 GPU에 모델을 로드합니다. 모델과 토크나이저를 개별적으로 살펴 보기 위해 따로따로 로드합니다.

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 모델과 토크나이저를 로드합니다.
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened.<|assistant|>"
prompt_kor = "사라에게 비극적인 정원사고에 대해 사과하는 이메일을 작성하고, 그것이 어떻게 일어났는지 설명하시오.<|assistant|>"

# 입력 프롬프트를 토큰으로 나눕니다.
input_ids = tokenizer(prompt_kor, return_tensors="pt").input_ids.to("cuda")

# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=60
)

# 출력을 프린트합니다.
print(tokenizer.decode(generation_output[0]))

사라에게 비극적인 정원사고에 대해 사과하는 이메일을 작성하고, 그것이 어떻게 일어났는지 설명하시오.<|assistant|> 제목: 저희 사원은 비극적인 정원사고에 대해 사과하고 싶습니다

안녕하세요, 


In [5]:
print(input_ids)

tensor([[29871, 30791, 31197, 31054,   237,   181,   143, 29871, 31487,   237,
           186,   188,   239,   163,   132, 30918, 29871, 30852, 31198, 30791,
         31137, 31054, 29871, 30890, 31435, 29871, 30791, 31906, 30944, 31081,
         29871, 30393,   238,   172,   151, 31153, 31286, 29871,   239,   161,
           148, 31126, 30944, 31137, 29892, 29871, 31607,   237,   181,   134,
         30393, 29871, 31129,   238,   153,   190,   237,   181,   143, 29871,
         31153, 31129,   238,   133,   175, 31081, 30811, 29871,   239,   135,
           167, 31976, 30944, 30889, 31346, 29889, 32001]], device='cuda:0')


In [6]:
for id in input_ids[0]:
   print(tokenizer.decode(id))


사
라
에
�
�
�

비
�
�
�
�
�
�
인

정
원
사
고
에

대
해

사
과
하
는

이
�
�
�
일
을

�
�
�
성
하
고
,

그
�
�
�
이

어
�
�
�
�
�
�

일
어
�
�
�
는
지

�
�
�
명
하
시
오
.
<|assistant|>


In [7]:
generation_output

tensor([[29871, 30791, 31197, 31054,   237,   181,   143, 29871, 31487,   237,
           186,   188,   239,   163,   132, 30918, 29871, 30852, 31198, 30791,
         31137, 31054, 29871, 30890, 31435, 29871, 30791, 31906, 30944, 31081,
         29871, 30393,   238,   172,   151, 31153, 31286, 29871,   239,   161,
           148, 31126, 30944, 31137, 29892, 29871, 31607,   237,   181,   134,
         30393, 29871, 31129,   238,   153,   190,   237,   181,   143, 29871,
         31153, 31129,   238,   133,   175, 31081, 30811, 29871,   239,   135,
           167, 31976, 30944, 30889, 31346, 29889, 32001, 29871, 31306,   238,
           173,   172, 29901, 29871,   239,   163,   131,   240,   160,   175,
         29871, 30791, 31198, 31354, 29871, 31487,   237,   186,   188,   239,
           163,   132, 30918, 29871, 30852, 31198, 30791, 31137, 31054, 29871,
         30890, 31435, 29871, 30791, 31906, 30944, 31137, 29871,   239,   142,
           185,   239,   141,   184, 31063, 30709,  

In [8]:
print(tokenizer.decode(3323))
print(tokenizer.decode(622))
print(tokenizer.decode([3323, 622]))
print(tokenizer.decode(29901))

Sub
ject
Subject
:


# 훈련된 LLM 토큰나이저 비교하기


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        # 텍스트를 인코딩한 후 다시 디코딩했을 때 원본 텍스트와 동일해지려면
        # clean_up_tokenization_spaces를 False로 지정해야 합니다.
        # 현재 이 매개변수의 기본값은 None(True에 해당)이며
        # transformers 4.45에서 True로 바뀔 예정입니다.
        # https://github.com/huggingface/transformers/issues/31884
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t, clean_up_tokenization_spaces=False) +
            '\x1b[0m',
            end=' '
        )

In [10]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"		" four spaces:"    "
12.0*50=600
"""

In [11]:
show_tokens(text, "bert-base-uncased")

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

In [12]:
show_tokens(text, "bert-base-cased")

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " four spaces : " " 12 . 0 * 50 = 600 [SEP] 

In [13]:
show_tokens(text, "gpt2")


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"        " 
 12 . 0 * 50 = 600 
 

In [14]:
show_tokens(text, "google/flan-t5-small")

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " four spaces : " " 12. 0 * 50 = 600  </s> 

In [15]:
# 공식 토크나이저는 `tiktoken`이지만 허깅 페이스 플랫폼에 동일한 토크나이저가 있습니다.
show_tokens(text, "Xenova/gpt-4")


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      "
 12 . 0 * 50 = 600 
 

In [16]:
show_tokens(text, "bigcode/starcoder2-15b")


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :" 	 	 "  four  spaces :"      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [17]:
show_tokens(text, "facebook/galactica-1.3b")


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : " 		 "  four  spaces : "      " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [18]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :" 	 	 " four spaces :"    " 
 1 2 . 0 * 5 0 = 6 0 0 
 

# (BERT와 같은) 언어 모델로 문맥을 고려한 단어 임베딩 만들기

In [19]:
from transformers import AutoModel, AutoTokenizer

# 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

# 언어 모델을 로드합니다.
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

text = "안녕 세상"

# 문장을 토큰으로 나눕니다.
tokens = tokenizer(text, return_tensors='pt')

# 토큰을 처리합니다.
output = model(**tokens)[0]

/usr/local/lib/python3.11/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [20]:
output.shape

torch.Size([1, 6, 384])

In [21]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))

[CLS]
안
녕
세
상
[SEP]


In [22]:
output

tensor([[[-3.3213e+00,  1.2678e-02, -1.0480e-02,  ..., -3.7859e-02,
          -1.9453e-01,  3.8344e-01],
         [-1.1770e+00, -5.7778e-02,  6.0894e-01,  ..., -1.3683e-01,
           5.5361e-01,  2.0429e-02],
         [-1.0397e+00, -2.6746e-01,  3.7950e-01,  ...,  2.9306e-01,
           2.1833e-01,  8.0596e-01],
         [-1.0819e+00,  2.9583e-02,  4.2122e-01,  ..., -2.5296e-01,
           3.3147e-01, -1.9402e-03],
         [-9.8954e-01, -2.3942e-01,  3.3319e-01,  ..., -2.7830e-01,
           3.5390e-02, -2.3211e-01],
         [-3.2431e+00,  1.0770e-01,  9.4596e-02,  ..., -1.8369e-03,
          -2.2409e-01,  1.2428e-01]]], grad_fn=<NativeLayerNormBackward0>)

# 텍스트 임베딩 (문장과 전체 문서)

In [12]:
from sentence_transformers import SentenceTransformer

# 모델을 로드합니다.
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 텍스트를 텍스트 임베딩으로 변환합니다.
vector = model.encode("Best movie ever!")
vector2 = model.encode("This is the best film I've ever seen.")
vector3 = model.encode("I like coffee.")

In [10]:
vector.shape

(768,)

In [15]:
import numpy as np

# Define a function to calculate cosine similarity
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

# Calculate and print the cosine similarities
print(f"Cosine similarity between vector and vector2: {cosine_similarity(vector, vector2)}")
print(f"Cosine similarity between vector and vector3: {cosine_similarity(vector, vector3)}")
print(f"Cosine similarity between vector and vector: {cosine_similarity(vector, vector)}")

Cosine similarity between vector and vector2: 0.6373659372329712
Cosine similarity between vector and vector3: 0.11804952472448349
Cosine similarity between vector and vector: 1.0


# LLM 밖의 단어 임베딩


In [16]:
# gensim을 설치합니다. 설치 후 코랩 런타임 세션을 다시 시작해 주세요.
!pip install gensim

In [17]:
import gensim.downloader as api

# 임베딩을 다운로드합니다 (66MB, glove, 위키백과에서 훈련됨, 벡터 크기: 50)
# "word2vec-google-news-300"도 선택 가능합니다.
# 더 자세한 옵션은 https://github.com/RaRe-Technologies/gensim-data 을 참고하세요.
model = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [24]:
# 일반적으로 bank 는 은행이라는 의미지만 강둑이라는 의미도 있다. 이 때는 word2vec 이 잘 동작할까?
model.most_similar([model['bank']], topn=20)

[('bank', 0.9999999403953552),
 ('banks', 0.869862973690033),
 ('securities', 0.7996813654899597),
 ('banking', 0.7965160012245178),
 ('investment', 0.7849708199501038),
 ('exchange', 0.7808825969696045),
 ('financial', 0.7670274972915649),
 ('credit', 0.764915406703949),
 ('lender', 0.7518407702445984),
 ('capital', 0.7380708456039429),
 ('brokerage', 0.7373986840248108),
 ('currency', 0.7363598346710205),
 ('investments', 0.7340353727340698),
 ('funds', 0.7284393906593323),
 ('fund', 0.7265523672103882),
 ('assets', 0.7237452864646912),
 ('loans', 0.7225799560546875),
 ('investors', 0.7135698795318604),
 ('equity', 0.7132648825645447),
 ('stock', 0.7128599286079407)]

# 임베딩으로 노래 추천하기

In [25]:
import pandas as pd
from urllib import request

# 재생목록 데이터셋 파일을 가져옵니다.
# 데이터셋 원본: https://www.cs.cornell.edu/~shuochen/lme/dataset.tar.gz
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# 재생목록 파일을 파싱합니다. 처음 두 줄은 메타데이터만 담고 있으므로 건너뜁니다.
lines = data.read().decode("utf-8").split('\n')[2:]

# 하나의 노래만 있는 재생목록은 삭제합니다.
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# 노래의 메타데이터를 로드합니다.
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [26]:
print('재생목록 #1:\n ', playlists[0], '\n')
print('재생목록 #2:\n ', playlists[1])

재생목록 #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

재생목록 #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117', '118', 

In [27]:
from gensim.models import Word2Vec

# Word2Vec 모델을 훈련합니다.
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)

In [28]:
song_id = 2172

# 노래 ID 2172와 비슷한 노래를 찾으라고 모델에게 요청합니다.
model.wv.most_similar(positive=str(song_id))

[('2849', 0.999363124370575),
 ('2976', 0.9982110261917114),
 ('2640', 0.9976083636283875),
 ('3094', 0.9974283576011658),
 ('5549', 0.9970372319221497),
 ('3136', 0.9966909289360046),
 ('2014', 0.9963603019714355),
 ('2715', 0.9961172938346863),
 ('6658', 0.9960623383522034),
 ('2704', 0.996033787727356)]

In [29]:
print(songs_df.iloc[2172])

title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [33]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# 추천 노래 출력
# 25475 	Autumn Leaves	Chet Baker
print_recommendations(25475)    # 나름 jazz 장르임

,title,artist
id,,
25530,"Use Me (w\/ Justo Almario, John Fumo & Courtne...",George Kahn
15353,Oh Really?,George Duke
43498,You Make Me Feel Brand New,Larry Carlton
20387,Midnight Blue,Kenny Burrell
45507,Always Together,Mr. Marcus Young


In [31]:
print_recommendations(2172)

,title,artist
id,,
2849,Run To The Hills,Iron Maiden
2976,I Don't Know,Ozzy Osbourne
2640,Red Barchetta,Rush
3094,Breaking The Law,Judas Priest
5549,November Rain,Guns N' Roses


In [32]:
print_recommendations(842)

,title,artist
id,,
12205,Give It Up To Me,Sean Paul
330,Hate It Or Love It (w\/ 50 Cent),The Game
886,Heartless,Kanye West
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
5668,How We Do (w\/ 50 Cent),The Game
